# Account-Level Bot Detection for #Silent Sam Tweets

## 0. Genreal Idea
This is a machine learning classification task. The basic idea is to train a model from labeled dataset and apply it to our user infomation.
## 1. Data Preparation
### 1.1 Scrapping more user information from Twitter

In [122]:
def get_name(user):
    userid=user.xpath('.//div[@class="UserActions   u-textLeft"]/div/@data-user-id')
    screenname=user.xpath('.//div[@class="UserActions   u-textLeft"]/div/@data-screen-name') 
    name=user.xpath('.//div[@class="UserActions   u-textLeft"]/div/@data-name')
    protected=user.xpath('.//div[@class="UserActions   u-textLeft"]/div/@data-protected')
    
    if userid==[]:
        userid=None
    elif screenname==[]:
        screenname=None
    elif name==[]:
        name=None
    elif protected==[]:
        protected=None
        
    return userid,screenname,name,protected

In [123]:
def get_card_info(user):
    boi=user.xpath('.//*[@class="ProfileHeaderCard-bio u-dir"]')
    if boi!=[] and boi[0].xpath("./text()")!=[]:
        boi=boi[0].xpath('string(.)')
    
    loc=user.xpath('.//span[@class="ProfileHeaderCard-locationText u-dir"]')
    if loc!=[] and loc[0].xpath(".//text()")!=[]:
        loc=loc[0].xpath('string(.)')
    
    url=user.xpath('.//span[@class="ProfileHeaderCard-urlText u-dir"]/a/@href')
    join_time=user.xpath('.//span[@class="ProfileHeaderCard-joinDateText js-tooltip u-dir"]/@title')
    
    if boi==[]:
        boi=None
    elif loc==[]:
        loc=None
    elif url==[]:
        url=None
    elif join_time==[]:
        join_time=None
    
    return boi,loc,url,join_time

In [124]:
def get_stats(user):
    tweets=user.xpath('.//*[@data-nav="tweets"]/span[@class="ProfileNav-value"]/@data-count')
    following=user.xpath('.//*[@data-nav="following"]/span[@class="ProfileNav-value"]/@data-count')
    followers=user.xpath('.//*[@data-nav="followers"]/span[@class="ProfileNav-value"]/@data-count')
    likes=user.xpath('.//*[@data-nav="favorites"]/span[@class="ProfileNav-value"]/@data-count')
    lists=user.xpath('.//*[@data-nav="all_lists"]/span[@class="ProfileNav-value"]//text()')
    #print(lists)
    if tweets==[]:
        tweets=None
    elif following==[]:
        following==None
    elif followers==[]:
        followers=None
    elif lists==[]:
        lists=None
    elif likes==[]:
        likes=None
    #don't know why some of them the elif commands don't work    
    if likes==[]:
        likes=None
        
    if lists==[]:
        likes=None
    return tweets,following,followers,likes,lists

In [125]:
def get_media(user):
    try:
        media_count = user.xpath('.//a[@class="PhotoRail-headingWithCount js-nav"]/text()')[0].strip().split(" ")[0]
        media_count = convertToInt(media_count)
    except:
        media_count = 0

    return [media_count]

In [126]:
def get_verified(user):
    try:
        is_verified = user.xpath('.//span[@class="ProfileHeaderCard-badges"]//text()')
        if "Verified account" in is_verified:
            is_verified = 1
        else:
            is_verified = 0
    except:
        is_verified = 0

    return [is_verified]

In [127]:
def convertToInt(x):

    multDict = {
        "k" : 1000,
        "m" : 1000000,
        "b" : 1000000000,
    }
    try:
        if ',' in x:
            x = x.replace(',', '')
        y = int(x)
        return y
    except:
        pass

    try:
        y = float(str(x)[:-1])
        y = y * multDict[str(x)[-1:].lower()]
        return int(y)
    except:
        pass

    return 0

In [128]:
def get_avatar(user):
    try:
        pic = user.xpath('.//*[@class="ProfileAvatar-image "]/@src')
 
    except:
        pic = None
    
    if pic==[]:
        pic=None
    
    return pic


In [130]:
def turn_null(d):
    for keys,values in d.items():
        if values==[]:
            d[keys]=None
    return d

In [131]:
def destroy_tree(tree):
    root = tree.getroot()

    node_tracker = {root: [0, None]}

    for node in root.iterdescendants():
        parent = node.getparent()
        node_tracker[node] = [node_tracker[parent][0] + 1, parent]

    node_tracker = sorted([(depth, parent, child) for child, (depth, parent)
                           in node_tracker.items()], key=lambda x: x[0], reverse=True)

    for _, parent, child in node_tracker:
        if parent is None:
            break
        parent.remove(child)

    del tree

In [132]:
def get_node(user_id):
    global i
    global j
    
    try:
        
        
        with requests.Session() as s:
            s.keep_alive = False
            resp=s.get(str(user_id),headers=headers,stream=False)
            content=resp.text
            resp.close()
            resp = None
            
            del(resp)
        #node=etree.HTML(content)
        #print(user_id)
        if i in j:
            print(i, "user are done")
        
        i=i+1
        
        
        #content=None
        gc.collect()
        return content
    except e:
        failed.append(user)
        print("failed")

In [2]:
import gc

import requests
import pandas as pd
import lxml
from fake_useragent import UserAgent
from lxml import etree
from io import StringIO, BytesIO

The user_db would be a dictionary to store every record of scrapped user information.


user_pool would be a list contains all user's names


failed and failed_add would be lists to store the users info that failed at scrapping



In [139]:
user_db={}
user_pool=[]
failed=[]


failed_add=[]

parser=etree.HTMLParser()

In [140]:
ram=pd.read_csv("Copy of Silent Sam Tweets - Archive.csv")['from_user'].map(lambda x:user_pool.append("http://twitter.com/"+str(x) ))

c:\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [141]:
urldb=pd.DataFrame(list(set(user_pool)), columns=["url"])

In [143]:
count=[i for i in range(1,len(user_pool),500)]

In [144]:
count

[1,
 501,
 1001,
 1501,
 2001,
 2501,
 3001,
 3501,
 4001,
 4501,
 5001,
 5501,
 6001,
 6501,
 7001,
 7501,
 8001,
 8501,
 9001,
 9501,
 10001,
 10501,
 11001,
 11501,
 12001,
 12501,
 13001,
 13501,
 14001,
 14501,
 15001,
 15501,
 16001,
 16501,
 17001,
 17501,
 18001,
 18501,
 19001,
 19501,
 20001,
 20501,
 21001,
 21501,
 22001,
 22501,
 23001,
 23501,
 24001,
 24501,
 25001,
 25501,
 26001,
 26501,
 27001,
 27501,
 28001,
 28501,
 29001,
 29501,
 30001,
 30501,
 31001,
 31501,
 32001,
 32501,
 33001,
 33501,
 34001,
 34501,
 35001,
 35501,
 36001,
 36501,
 37001,
 37501,
 38001,
 38501,
 39001,
 39501,
 40001,
 40501,
 41001,
 41501,
 42001,
 42501,
 43001,
 43501,
 44001,
 44501,
 45001,
 45501,
 46001,
 46501,
 47001,
 47501,
 48001,
 48501,
 49001,
 49501,
 50001,
 50501,
 51001,
 51501,
 52001,
 52501,
 53001,
 53501,
 54001,
 54501,
 55001,
 55501,
 56001,
 56501,
 57001,
 57501,
 58001,
 58501,
 59001,
 59501,
 60001,
 60501,
 61001,
 61501,
 62001,
 62501,
 63001,
 63501,


In [149]:
i=0;
for user_id in urldb['url']:
    try:
        print(user_id)
        user=etree.parse(StringIO(get_node(user_id)),parser)
        
        
    except e:
        failed.append(user_id)
        print("failed")
    
    
    
    names=get_name(user)
    cards=get_card_info(user)
    stats=get_stats(user)
    
    current_user={'userid':names[0],
            'screenname':names[1],
            'name':names[2],
            'proctected':names[3],
            'boi':cards[0],
            'loc':cards[1],
            'url':cards[2],
            'join_time':cards[3],
            'tweets':stats[0],
            'following':stats[1],
            'followers':stats[2],
            'likes':stats[3],
            'lists':stats[4],
            'media':get_media(user),
            'verified':get_verified(user),
             'avatar':get_avatar(user)}
    
    #print(current_user)
    try:
        current_user=turn_null(current_user)
        user_db[i]=current_user
        print(i,"user's info has been added")
        #i+=1
        current_user=None
        #user=None
        gc.collect()
        destroy_tree(user)
#         if i in count:
#             print(i,"records has been added")
    except:
        failed_add.append(current_user)
        current_user=None
        user=None
        gc.collect()
        print(current_user)

http://twitter.com/corrotta
1 user's info has been added
http://twitter.com/Breaking_NewsTN
2 user's info has been added
http://twitter.com/Owd_Mueller
3 user's info has been added
http://twitter.com/superbigrobot
4 user's info has been added
http://twitter.com/Mushy
5 user's info has been added
http://twitter.com/tice_stacy
6 user's info has been added
http://twitter.com/FitzFortune
7 user's info has been added
http://twitter.com/Polk_stoner
8 user's info has been added
http://twitter.com/AlehousePhoto
9 user's info has been added
http://twitter.com/JodiAll77904488
10 user's info has been added
http://twitter.com/Wine_Pres
11 user's info has been added
http://twitter.com/brione
12 user's info has been added
http://twitter.com/BreitSteve
13 user's info has been added
http://twitter.com/dkulchar
14 user's info has been added
http://twitter.com/DisallieWalker
15 user's info has been added
http://twitter.com/Aurania101
16 user's info has been added
http://twitter.com/carolnfoster
17 user'

135 user's info has been added
http://twitter.com/sentientsloot
136 user's info has been added
http://twitter.com/DFost14
137 user's info has been added
http://twitter.com/vannabbott
138 user's info has been added
http://twitter.com/23brix
139 user's info has been added
http://twitter.com/misajane79
140 user's info has been added
http://twitter.com/connortheOGfam
141 user's info has been added
http://twitter.com/pegele03
142 user's info has been added
http://twitter.com/StraightUpUSA
143 user's info has been added
http://twitter.com/JoeKoffee
144 user's info has been added
http://twitter.com/wasa3lyyy
145 user's info has been added
http://twitter.com/waliqamara
146 user's info has been added
http://twitter.com/epiktistes
147 user's info has been added
http://twitter.com/willishl
148 user's info has been added
http://twitter.com/RedNationRising
149 user's info has been added
http://twitter.com/barneyray1
150 user's info has been added
http://twitter.com/o_guest
151 user's info has been 

269 user's info has been added
http://twitter.com/ShoeDaydreams
270 user's info has been added
http://twitter.com/DTWriters
271 user's info has been added
http://twitter.com/OvertonMargaret
272 user's info has been added
http://twitter.com/WanderingEgo
273 user's info has been added
http://twitter.com/pterantula
274 user's info has been added
http://twitter.com/solarblaster1
275 user's info has been added
http://twitter.com/mattias_jonsson
276 user's info has been added
http://twitter.com/StphanieTheBest
277 user's info has been added
http://twitter.com/ilija34
278 user's info has been added
http://twitter.com/wordnerdy
279 user's info has been added
http://twitter.com/Strwar11
280 user's info has been added
http://twitter.com/java_momma19
281 user's info has been added
http://twitter.com/AlmaJMcCarty
282 user's info has been added
http://twitter.com/BirchCelloWren
283 user's info has been added
http://twitter.com/ChristineHWhyte
284 user's info has been added
http://twitter.com/AfarBe

402 user's info has been added
http://twitter.com/TonyByrne
403 user's info has been added
http://twitter.com/gamerguy1991
404 user's info has been added
http://twitter.com/SofieFleischer
405 user's info has been added
http://twitter.com/intergalactikK
406 user's info has been added
http://twitter.com/tomgreen1959
407 user's info has been added
http://twitter.com/kentomcdonald
408 user's info has been added
http://twitter.com/StormTroop333
409 user's info has been added
http://twitter.com/AntheaDCharles
410 user's info has been added
http://twitter.com/__jurnee
411 user's info has been added
http://twitter.com/AnarchoSaganist
412 user's info has been added
http://twitter.com/SamuelBass
413 user's info has been added
http://twitter.com/fuckboiseth
414 user's info has been added
http://twitter.com/frozencemetery
415 user's info has been added
http://twitter.com/Bravonavo
416 user's info has been added
http://twitter.com/Mandolin_75
417 user's info has been added
http://twitter.com/ISalas

534 user's info has been added
http://twitter.com/john_zealand
535 user's info has been added
http://twitter.com/Anonymo06640619
536 user's info has been added
http://twitter.com/krimetysakes
537 user's info has been added
http://twitter.com/AzAnComm
538 user's info has been added
http://twitter.com/1rishinfidel
539 user's info has been added
http://twitter.com/mmasterflex
540 user's info has been added
http://twitter.com/Pensive_JC
541 user's info has been added
http://twitter.com/cblaketurner
542 user's info has been added
http://twitter.com/janeenn1231
543 user's info has been added
http://twitter.com/fiestamule
544 user's info has been added
http://twitter.com/letssTalkCHELSI
545 user's info has been added
http://twitter.com/whitneyradar
546 user's info has been added
http://twitter.com/__weirdality_
547 user's info has been added
http://twitter.com/rorygee8
548 user's info has been added
http://twitter.com/peaches12348
549 user's info has been added
http://twitter.com/RedFlannelPu

...

29043 user's info has been added
http://twitter.com/_alexisdrake_
29044 user's info has been added
http://twitter.com/JasonShoultz
29045 user's info has been added
http://twitter.com/MaryinDurham
29046 user's info has been added
http://twitter.com/BlueTexGal
29047 user's info has been added
http://twitter.com/IsaacHale
29048 user's info has been added
http://twitter.com/jonibraz
29049 user's info has been added
http://twitter.com/sydneypersing
29050 user's info has been added
http://twitter.com/JoannaDeLaune
29051 user's info has been added
http://twitter.com/taskerrrr
29052 user's info has been added
http://twitter.com/tracydeonn
29053 user's info has been added
http://twitter.com/ASAPVick_
29054 user's info has been added
http://twitter.com/rebeccakreitzer
29055 user's info has been added
http://twitter.com/TheSaigonKid
29056 user's info has been added
http://twitter.com/cwharlow
29057 user's info has been added
http://twitter.com/StetsonKinnison
29058 user's info has been added
http

29171 user's info has been added
http://twitter.com/WomanVote
29172 user's info has been added
http://twitter.com/reallyverytired
29173 user's info has been added
http://twitter.com/jaysabur
29174 user's info has been added
http://twitter.com/HypeNC
29175 user's info has been added
http://twitter.com/RevVictoryLime
29176 user's info has been added
http://twitter.com/BraceFace_Ryse
29177 user's info has been added
http://twitter.com/CaptFlorida
29178 user's info has been added
http://twitter.com/Padilla_Lesliee
29179 user's info has been added
http://twitter.com/slsmnds
29180 user's info has been added
http://twitter.com/twilight2_0
29181 user's info has been added
http://twitter.com/AmeliaTheGingey
29182 user's info has been added
http://twitter.com/BernieDoesIt
29183 user's info has been added
http://twitter.com/JheeraBlade
29184 user's info has been added
http://twitter.com/willmasonmusic
29185 user's info has been added
http://twitter.com/cdzapolsky1
29186 user's info has been added

29299 user's info has been added
http://twitter.com/commanderwhith
29300 user's info has been added
http://twitter.com/NathanAngelus
29301 user's info has been added
http://twitter.com/BaldwinVidal
29302 user's info has been added
http://twitter.com/DCGinnie
29303 user's info has been added
http://twitter.com/melinasian
29304 user's info has been added
http://twitter.com/Thooleramawn
29305 user's info has been added
http://twitter.com/efemmera
29306 user's info has been added
http://twitter.com/CarlKenner
29307 user's info has been added
http://twitter.com/Pissed_Woman
29308 user's info has been added
http://twitter.com/HouseOfDowd
29309 user's info has been added
http://twitter.com/sj_vet45
29310 user's info has been added
http://twitter.com/DoraLeland
29311 user's info has been added
http://twitter.com/SarcasticMarx
29312 user's info has been added
http://twitter.com/_lowkeyKat
29313 user's info has been added
http://twitter.com/chathamnc
29314 user's info has been added
http://twitt

http://twitter.com/young_padewan_7
29427 user's info has been added
http://twitter.com/DocHolladayD
29428 user's info has been added
http://twitter.com/ColleenKelley1
29429 user's info has been added
http://twitter.com/aymanfadel
29430 user's info has been added
http://twitter.com/DoreenPalmieri
29431 user's info has been added
http://twitter.com/dbgill
29432 user's info has been added
http://twitter.com/knytych3
29433 user's info has been added
http://twitter.com/carsondial
29434 user's info has been added
http://twitter.com/PlanchatCubana
29435 user's info has been added
http://twitter.com/corinnegoud
29436 user's info has been added
http://twitter.com/XOXDi
29437 user's info has been added
http://twitter.com/gabrielleiman_
29438 user's info has been added
http://twitter.com/415tothe919
29439 user's info has been added
http://twitter.com/kellyimca
29440 user's info has been added
http://twitter.com/LettersBookshop
29441 user's info has been added
http://twitter.com/GodUWish
29442 use

29554 user's info has been added
http://twitter.com/allyd682
29555 user's info has been added
http://twitter.com/cruzn4me03
29556 user's info has been added
http://twitter.com/YeahBee_
29557 user's info has been added
http://twitter.com/Scrubbywhiteguy
29558 user's info has been added
http://twitter.com/JTRipper121
29559 user's info has been added
http://twitter.com/leewm2012
29560 user's info has been added
http://twitter.com/slarote
29561 user's info has been added
http://twitter.com/marcdglenn
29562 user's info has been added
http://twitter.com/JohnBel20027170
29563 user's info has been added
http://twitter.com/TheRealSerdadu
29564 user's info has been added
http://twitter.com/jeshuf
29565 user's info has been added
http://twitter.com/shepazu
29566 user's info has been added
http://twitter.com/zannibunny
29567 user's info has been added
http://twitter.com/Schmaniel
29568 user's info has been added
http://twitter.com/teamwhiteboys_
29569 user's info has been added
http://twitter.com/

In [151]:
user_db

{1: {'userid': None,
  'screenname': None,
  'name': None,
  'proctected': None,
  'boi': None,
  'loc': None,
  'url': None,
  'join_time': None,
  'tweets': None,
  'following': None,
  'followers': None,
  'likes': None,
  'lists': None,
  'media': [0],
  'verified': [0],
  'avatar': None},
 3: {'userid': ['433586880'],
  'screenname': ['Owd_Mueller'],
  'name': ['Scott I Miss the Sun Mueller'],
  'proctected': ['false'],
  'boi': 'Relocated Kansan. Father. Facts and science are important. So is compassion and understanding. #BlueWave',
  'loc': '\n              Wisconsin, USA\n\n        ',
  'url': None,
  'join_time': ['11:11 AM - 10 Dec 2011'],
  'tweets': ['1773'],
  'following': ['404'],
  'followers': ['40'],
  'likes': ['1732'],
  'lists': None,
  'media': [53],
  'verified': [0],
  'avatar': ['https://pbs.twimg.com/profile_images/1080303580903485440/388ISZrJ_400x400.jpg']},
 5: {'userid': ['8124242'],
  'screenname': ['Mushy'],
  'name': ['Mauricio (Mushi)'],
  'proctected':

In [150]:
len(user_db)

29639

In [157]:
user_pd=pd.DataFrame(user_db).T

In [158]:
user_pd.to_csv("user_pd1.csv")

In [211]:
user_pd['name_none']=user_pd['name'].map(lambda x: x==None)

In [266]:
clean1=user_pd[user_pd['name_none']==False]

In [268]:
clean1.loc[:,'followers':'lists']=clean1.loc[:,'followers':'lists'].applymap(lambda x:x[0] if x!=None else x)

c:\python36\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [269]:
clean1.loc[:,'media':"verified"]=clean1.loc[:,'media':"verified"].applymap(lambda x:x[0] if x!=None else x)

c:\python36\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [271]:
clean1.proctected=clean1.proctected.map(lambda x: 0 if x=="false" else 1)

c:\python36\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [277]:
clean1.proctected=clean1.proctected.astype("bool")

c:\python36\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [280]:
clean1.verified=clean1.verified.astype("bool")

c:\python36\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [288]:
clean1['loc']=clean1['loc'].map(lambda x: x.strip())

c:\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [292]:
clean1.avatar=clean1.avatar.map(lambda x:x[0] if x!=None else x)

c:\python36\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [347]:
clean1['join_time']=pd.to_datetime(clean1['join_time'],infer_datetime_format=True, format="%I:%M %p - %d %b %Y")

c:\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [390]:
clean1.boi=clean1.boi.map(lambda x: None if type(x)==list else x)

c:\python36\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [393]:
clean1.lists=clean1.lists.fillna(0)

c:\python36\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [396]:
clean2=clean1.drop(columns="name_none")

In [419]:
clean2['has_bpi']=clean2['boi'].map(lambda x: False if x==None else True)

In [425]:
clean2['has_loc']=clean2['loc'].map(lambda x: False if len(x)==0 else True)

In [426]:
clean2['has_url']=clean2['url'].map(lambda x: False if x==None else True)

In [465]:
import re
import numpy as np
clean2['def_ava']=clean2['avatar'].map(lambda x: True if re.search(r"default",str(x))!=None else False)


In [527]:
clean3=clean2.loc[:,['name','followers',"following","join_time","lists",'protected','screenname','tweets','likes','verified','has_bpi','has_loc','has_url','def_ava']]

c:\python36\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [528]:
clean3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26929 entries, 3 to 29639
Data columns (total 14 columns):
name          26929 non-null object
followers     26869 non-null object
following     26844 non-null object
join_time     26929 non-null datetime64[ns]
lists         26929 non-null object
protected     0 non-null float64
screenname    26929 non-null object
tweets        26886 non-null object
likes         26748 non-null object
verified      26929 non-null bool
has_bpi       26929 non-null bool
has_loc       26929 non-null bool
has_url       26929 non-null bool
def_ava       26929 non-null bool
dtypes: bool(5), datetime64[ns](1), float64(1), object(7)
memory usage: 3.4+ MB


In [531]:
clean3.columns=['name', 'followers', 'following', 'join_time', 'lists', 'protected',
       'screenname', 'tweets', 'likes', 'verified', 'has_boi', 'has_loc',
       'has_url', 'def_ava']

,name,followers,following,join_time,lists,protected,screenname,tweets,likes,verified,has_boi,has_loc,has_url,def_ava
3,Scott I Miss the Sun Mueller,40,404,2011-12-10 11:11:00,0,NaN,Owd_Mueller,1773,1732,False,True,True,False,False
5,Mauricio (Mushi),565,557,2007-08-11 07:07:00,5,NaN,Mushy,16666,191,False,True,True,True,False
7,Fitz Fortune,1021,2407,2008-12-16 02:50:00,0,NaN,FitzFortune,9567,21874,False,True,True,False,False
9,Alehouse Photography,106,517,2011-06-01 19:40:00,0,NaN,AlehousePhoto,3692,6097,False,True,True,True,False
11,Presley,245,336,2011-03-24 20:00:00,0,NaN,Wine_Pres,6680,28917,False,False,True,False,False
13,Steve Breit,145,250,2013-04-13 15:04:00,0,NaN,BreitSteve,3391,7559,False,True,True,False,False
15,MOMMARED,952,1075,2018-02-06 08:21:00,0,NaN,DisallieWalker,8412,14809,False,True,False,False,False
17,Carolyn Foster,868,2893,2008-11-07 01:27:00,5,NaN,carolnfoster,69533,19362,False,True,True,False,False
19,Christopher Suprun,13452,846,2014-11-04 05:57:00,0,NaN,TheChrisSuprun,46617,32,True,True,True,False,False
21,Uncle Sam,198,265,2017-08-18 17:55:00,0,NaN,SamuelScottElm1,1614,2021,False,True,True,False,False


In [ ]:
test_delta=datetime.datetime.now()-clean3.loc[:,'join_time']
clean3['join_time']=test_delta.astype('timedelta64[D]')

In [622]:
clean3.columns.tolist()

['name',
 'followers',
 'following',
 'join_time',
 'lists',
 'protected',
 'screenname',
 'tweets',
 'likes',
 'verified',
 'has_boi',
 'has_loc',
 'has_url',
 'def_ava']

In [624]:
new_colis=['name', 'screenname',
 'followers',
 'following',
 'join_time',
 'lists',
 'protected',

 'tweets',
 'likes',
 'verified',
 'has_boi',
 'has_loc',
 'has_url',
 'def_ava']

In [628]:
clean3=clean3.loc[:,new_colis]

In [631]:
clean3.protected=clean3.protected.notna()

In [636]:
clean3=clean3.reset_index().drop(columns="index")

In [17]:
clean3.iloc[:,2:]=clean3.iloc[:,2:].fillna(0)

In [708]:
clean3.to_csv("to_be_test.csv")

In [307]:
bots1=pd.read_csv(r'C:\Users\Jayden x Reich\Desktop\Digital Innovation Lab\social_spambots_1.csv\users.csv')

In [398]:
bots2=pd.read_csv(r'C:\Users\Jayden x Reich\Desktop\Digital Innovation Lab\social_spambots_2.csv\users.csv')

In [399]:
bots3=pd.read_csv(r'C:\Users\Jayden x Reich\Desktop\Digital Innovation Lab\social_spambots_3.csv\users.csv')

In [401]:
trabot1=pd.read_csv(r'C:\Users\Jayden x Reich\Desktop\Digital Innovation Lab\traditional_spambots_1.csv\users.csv')

In [402]:
trabot2=pd.read_csv(r'C:\Users\Jayden x Reich\Desktop\Digital Innovation Lab\traditional_spambots_2.csv\users.csv')

In [403]:
trabot3=pd.read_csv(r'C:\Users\Jayden x Reich\Desktop\Digital Innovation Lab\traditional_spambots_3.csv\users.csv')

In [404]:
trabot4=pd.read_csv(r'C:\Users\Jayden x Reich\Desktop\Digital Innovation Lab\traditional_spambots_4.csv\users.csv')

In [408]:
allbots=pd.concat([bots1,bots2,bots3,trabot1,trabot2,trabot3,trabot4])

c:\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [411]:
allbots=allbots.reset_index().drop(columns="index")

In [458]:
allbots['timestamp']=pd.to_datetime(allbots['timestamp'])

In [507]:
allbots_clean1=allbots.loc[:,['name','description','favourites_count','followers_count','friends_count','listed_count','protected','verified','timestamp','url','statuses_count','screen_name','profile_image_url','location']]

In [508]:
allbots_clean1["has_boi"]=allbots_clean1.description.notna()

In [509]:
allbots_clean1.protected=allbots_clean1.protected.notna()

In [510]:
allbots_clean1.verified=allbots_clean1.verified.notna()

In [511]:
allbots_clean1['has_url']=allbots_clean1.url.notna()

In [513]:
allbots_clean1['has_loc']=allbots_clean1.location.notna()

In [516]:
allbots_clean1['def_ava']=allbots_clean1.profile_image_url.map(lambda x: True if re.search(r"default",str(x))!=None else False)

In [520]:
allbots_clean2=allbots_clean1.loc[:,['name','favourites_count','followers_count',"friends_count","listed_count","protected","verified","timestamp","statuses_count",
                     "screen_name","has_boi","has_url","has_loc","def_ava"]]

In [524]:
allbots_clean2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7543 entries, 0 to 7542
Data columns (total 14 columns):
name                7543 non-null object
favourites_count    7543 non-null int64
followers_count     7543 non-null int64
friends_count       7543 non-null int64
listed_count        7543 non-null int64
protected           7543 non-null bool
verified            7543 non-null bool
timestamp           7543 non-null datetime64[ns]
statuses_count      7543 non-null int64
screen_name         7543 non-null object
has_boi             7543 non-null bool
has_url             7543 non-null bool
has_loc             7543 non-null bool
def_ava             7543 non-null bool
dtypes: bool(6), datetime64[ns](1), int64(5), object(2)
memory usage: 515.7+ KB


In [554]:
allbots_clean2.columns=['name', 'likes', 'followers', 'fowllowing',
       'listst', 'protected', 'verified', 'join_time', 'tweets',
       'screenname', 'has_boi', 'has_url', 'has_loc', 'def_ava']

In [555]:
allbots_clean2['is_bot']=True

In [536]:
real=pd.read_csv(r'C:\Users\Jayden x Reich\Desktop\Digital Innovation Lab\genuine_accounts.csv\users.csv')

In [537]:
real['timestamp']=pd.to_datetime(real['timestamp'])

In [538]:
real_clean1=real.loc[:,['name','description','favourites_count','followers_count','friends_count','listed_count','protected','verified','timestamp','url','statuses_count','screen_name','profile_image_url','location']]

In [539]:
real_clean1["has_boi"]=real_clean1.description.notna()
real_clean1.protected=real_clean1.protected.notna()
real_clean1.verified=real_clean1.verified.notna()
real_clean1['has_url']=real_clean1.url.notna()
real_clean1['has_loc']=real_clean1.location.notna()
real_clean1['def_ava']=real_clean1.profile_image_url.map(lambda x: True if re.search(r"default",str(x))!=None else False)
real_clean2=real_clean1.loc[:,['name','favourites_count','followers_count',"friends_count","listed_count","protected","verified","timestamp","statuses_count",
                     "screen_name","has_boi","has_url","has_loc","def_ava"]]
real_clean2.columns=['name', 'likes', 'followers', 'fowllowing',
       'listst', 'protected', 'verified', 'join_time', 'tweets',
       'screenname', 'has_boi', 'has_url', 'has_loc', 'def_ava']

In [541]:
real_clean2["is_bot"]=False

In [557]:
test_db=pd.concat([real_clean2,allbots_clean2])

In [559]:
test_db=test_db.reset_index().drop(columns="index")

In [572]:
import datetime
datetime.datetime.now()

datetime.datetime(2019, 2, 24, 20, 7, 30, 600967)

In [595]:
import time
delta=datetime.datetime.now()-test_db.loc[:,'join_time']

In [597]:
test_db['join_time']=delta.astype('timedelta64[D]')

In [614]:
test_db.to_csv('test_db.csv')

In [604]:
test_db.columns.tolist()

['name',
 'likes',
 'followers',
 'fowllowing',
 'listst',
 'protected',
 'verified',
 'join_time',
 'tweets',
 'screenname',
 'has_boi',
 'has_url',
 'has_loc',
 'def_ava',
 'is_bot']

In [606]:
col_order=['name', 'screenname',
 'likes',
 'followers',
 'fowllowing',
 'listst',
 'protected',
 'verified',
 'join_time',
 'tweets',

 'has_boi',
 'has_url',
 'has_loc',
 'def_ava',
 'is_bot']

In [608]:
test_db=test_db[col_order]

In [751]:
test_db.to_csv("test_db.csv")

## 2. Machine Learning

In [3]:
test_db=pd.read_csv("test_db.csv")

In [11]:
test_db=test_db.drop(columns="Unnamed: 0")

In [4]:
clean3=pd.read_csv("to_be_test.csv")

In [13]:
clean3=clean3.drop(columns="Unnamed: 0")

In [14]:
from sklearn.model_selection import cross_val_score
# Training data features, skip the first column 'Survived'
train_features = test_db.iloc[:, 2:-1]

# 'Survived' column values
train_target = test_db.iloc[:,-1]

# Fit the model to our training data
clf = clf.fit(train_features, train_target)
score = clf.score(train_features, train_target)
"Mean accuracy of Random Forest: {0}".format(score)

cross_val_score(clf, train_features, train_target, cv = 10, scoring='precision')

array([0.98931909, 0.98255034, 0.98051948, 0.98548813, 0.98427261,
       0.9867374 , 0.99052774, 0.98026316, 0.97916667, 0.98691099])

In [752]:
clf.feature_importances_

array([5.45389469e-01, 7.50754139e-02, 7.41107166e-02, 1.65803790e-02,
       7.30022706e-04, 1.32142661e-04, 4.62978830e-02, 1.86918723e-01,
       2.73498034e-02, 7.75118260e-03, 1.91364599e-02, 5.27803804e-04])

In [ ]:
# Feature Selection

In [744]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
rfe = RFE(model, 10)
fit = rfe.fit(train_features, train_target)
#print("Num Features: %d") % fit.n_features_
#print("Selected Features: %s") % fit.support_
#print("Feature Ranking: %s") % fit.ranking_

In [5]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

c:\python36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [771]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': range(10,101,10)

   
             }


grid_clf = GridSearchCV(clf, param_grid, cv=10)
grid_clf.fit(train_features, train_target)
grid_clf.best_params_

{'n_estimators': 60}

In [773]:
from sklearn.model_selection import GridSearchCV
clf = RandomForestClassifier(n_estimators= 100)
param_grid = { 'min_samples_split':range(80,150,20), 
    'min_samples_leaf':range(10,60,10),
   
             }


grid_clf = GridSearchCV(clf, param_grid, cv=10)
grid_clf.fit(train_features, train_target)
grid_clf.best_params_

{'min_samples_leaf': 20, 'min_samples_split': 120}

In [776]:
 from sklearn.model_selection import GridSearchCV
clf = RandomForestClassifier(n_estimators=100, min_samples_leaf=20)
param_grid = {   'max_depth':range(3,14,2), 
        'min_samples_split':range(50,201,20)
   
             }


grid_clf = GridSearchCV(clf, param_grid, cv=10)
grid_clf.fit(train_features, train_target)
grid_clf.best_params_


{'max_depth': 9, 'min_samples_split': 90}

In [15]:
clf = RandomForestClassifier(n_estimators=100, min_samples_leaf=20, max_depth=9, min_samples_split=90)

In [19]:
clf.fit(train_features, train_target)
clf.predict(clean3.iloc[:,2:])
import collections
collections.Counter(clf.predict(clean3.iloc[:,2:]))

Counter({False: 26633, True: 296})

In [ ]:
clf.predict(clean3.iloc[:,2:-1]).

In [21]:
clean3['is_bot_ramfor']=clf.predict(clean3.iloc[:,2:])

In [22]:
from sklearn.svm import SVC # "Support Vector Classifier"
clf_svc = SVC(kernel='linear')

In [23]:
clf_svc = clf_svc.fit(train_features, train_target)

KeyboardInterrupt: 

In [ ]:
clean3["is_bot_svc"]=clf_svc.predict(clean3.iloc[:,2:])

In [ ]:
clean3.is_bot_svc.value_counts()

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.learning_curve import validation_curve
import numpy as np



from sklearn.model_selection import GridSearchCV
logreg = LogisticRegression()
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}


grid_clf = GridSearchCV(logreg,grid,cv=10)
grid_clf.fit(train_features, train_target)
grid_clf.best_score_

0.8890805119360987

In [35]:
logre=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [36]:
logre.fit(train_features,train_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [39]:
clean3["is_bot_log"]=logre.predict(clean3.iloc[:,2:-1])

In [42]:
collections.Counter(logre.predict(clean3.iloc[:,2:-2]))

Counter({False: 26684, True: 245})

In [94]:
from sklearn.neural_network import MLPClassifier  
mlp = MLPClassifier(hidden_layer_sizes=(10, 10,10,100,100), max_iter=1000)  
mlp.fit(train_features, train_target)  

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10, 10, 100, 1000),
       learning_rate='constant', learning_rate_init=0.001, max_iter=1000,
       momentum=0.9, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [95]:
mlp.score(train_features, train_target)

0.9307433965689389

In [96]:
clean3["is_bot_dl"]=mlp.predict(clean3.iloc[:,2:-3])

In [97]:
clean3.is_bot_dl.value_counts()

False    22934
True      3995
Name: is_bot_dl, dtype: int64

## 3. Merge 2 dataset together

In [101]:
tweets_db=pd.read_csv("Copy of Silent Sam Tweets - Archive.csv")

c:\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [103]:
clean3

,name,screenname,followers,following,join_time,lists,protected,tweets,likes,verified,has_boi,has_loc,has_url,def_ava,is_bot_ramfor,is_bot_log,is_bot_dl
0,Scott I Miss the Sun Mueller,Owd_Mueller,40.0,404.0,2633.0,0,False,1773.0,1732.0,False,True,True,False,False,False,False,False
1,Mauricio (Mushi),Mushy,565.0,557.0,4215.0,5,False,16666.0,191.0,False,True,True,True,False,False,False,False
2,Fitz Fortune,FitzFortune,1021.0,2407.0,3722.0,0,False,9567.0,21874.0,False,True,True,False,False,False,False,False
3,Alehouse Photography,AlehousePhoto,106.0,517.0,2825.0,0,False,3692.0,6097.0,False,True,True,True,False,False,False,False
4,Presley,Wine_Pres,245.0,336.0,2894.0,0,False,6680.0,28917.0,False,False,True,False,False,False,False,False
5,Steve Breit,BreitSteve,145.0,250.0,2143.0,0,False,3391.0,7559.0,False,True,True,False,False,False,False,False
6,MOMMARED,DisallieWalker,952.0,1075.0,383.0,0,False,8412.0,14809.0,False,True,False,False,False,False,False,False
7,Carolyn Foster,carolnfoster,868.0,2893.0,3761.0,5,False,69533.0,19362.0,False,True,True,False,False,False,False,False
8,Christopher Suprun,TheChrisSuprun,13452.0,846.0,1573.0,0,False,46617.0,32.0,True,True,True,False,False,False,False,False
9,Uncle Sam,SamuelScottElm1,198.0,265.0,555.0,0,False,1614.0,2021.0,False,True,True,False,False,False,False,False


In [111]:
tweets_detected=pd.merge(tweets_db, clean3.loc[:, :], how='left',left_on="from_user", right_on="screenname")

In [126]:
tweets_detected[(tweets_detected["is_bot_ramfor"]==True) | (tweets_detected["is_bot_log"]==True)]

,id_str,from_user,text,created_at,time,geo_coordinates,user_lang,in_reply_to_user_id_str,in_reply_to_screen_name,from_user_id_str,...,tweets,likes,verified,has_boi,has_loc,has_url,def_ava,is_bot_ramfor,is_bot_log,is_bot_dl
290,1.093916e+18,isiscommodity,RT @RaleighReporter: Probably isn't surprising...,Fri Feb 08 16:55:46 +0000 2019,08/02/2019 16:55:46,NaN,en,NaN,NaN,1.088647e+18,...,111.0,130.0,False,True,True,True,False,True,False,False
376,1.093678e+18,JeffTho56961358,RT @RaleighReporter: Probably isn't surprising...,Fri Feb 08 01:10:30 +0000 2019,08/02/2019 01:10:30,NaN,en,NaN,NaN,1.066253e+18,...,37.0,0.0,False,False,False,False,False,True,True,True
377,1.093678e+18,JeffTho56961358,RT @RaleighReporter: Probably isn't surprising...,Fri Feb 08 01:10:30 +0000 2019,08/02/2019 01:10:30,NaN,en,NaN,NaN,1.066253e+18,...,37.0,0.0,False,False,False,False,False,True,True,True
663,1.093529e+18,Hashtaganalyze1,Top Trends of #USA in last 24 hours:\n\n #Divi...,Thu Feb 07 15:16:05 +0000 2019,07/02/2019 15:16:05,NaN,en,NaN,NaN,1.092688e+18,...,17.0,7.0,False,True,True,False,False,True,True,True
714,1.093514e+18,theIightcame,RT @RaleighReporter: Probably isn't surprising...,Thu Feb 07 14:16:17 +0000 2019,07/02/2019 14:16:17,NaN,en,NaN,NaN,2.176321e+09,...,9.0,38.0,False,True,False,False,False,True,True,True
1005,1.093356e+18,JNikonchuk,RT @RaleighReporter: Probably isn't surprising...,Thu Feb 07 03:47:52 +0000 2019,07/02/2019 03:47:52,NaN,en,NaN,NaN,2.707205e+09,...,337.0,176.0,False,False,True,False,False,True,False,True
1026,1.093349e+18,JeremyGlassford,RT @RaleighReporter: Probably isn't surprising...,Thu Feb 07 03:23:04 +0000 2019,07/02/2019 03:23:04,NaN,en,NaN,NaN,7.746952e+08,...,33.0,0.0,False,True,True,False,False,True,True,True
1231,1.093311e+18,mariebarkly,RT @RaleighReporter: Probably isn't surprising...,Thu Feb 07 00:48:45 +0000 2019,07/02/2019 00:48:45,NaN,en,NaN,NaN,9.794890e+17,...,715.0,2138.0,False,True,True,False,False,True,False,False
1240,1.093309e+18,kyudoka1,RT @RaleighReporter: Probably isn't surprising...,Thu Feb 07 00:44:08 +0000 2019,07/02/2019 00:44:08,NaN,en,NaN,NaN,4.251242e+09,...,799.0,886.0,False,False,False,False,False,True,False,False
1346,1.093295e+18,BlueWaveParty1,RT @RaleighReporter: Probably isn't surprising...,Wed Feb 06 23:46:59 +0000 2019,06/02/2019 23:46:59,NaN,en,NaN,NaN,1.014531e+18,...,28.0,20.0,False,True,True,False,False,True,True,True


In [112]:
tweets_detected.to_csv("tweets_detected.csv")

In [99]:
result=clean3

In [100]:
result.to_csv("result.csv")

In [729]:
test_y = clf.predict(clean3.iloc[:,2:])

In [734]:
clean3.is_bot.value_counts()

False    19021
True      7908
Name: is_bot, dtype: int64

In [218]:
#for testing

In [19]:
ua = UserAgent()
headers = {'User-Agent': ua.random}

In [103]:
get_node(urldb['url'][1])

b'<!DOCTYPE html>\n<html lang="en" data-scribe-reduced-action-queue="true">\n  <head>\n    \n    \n    \n    \n    \n    \n    \n    <meta charset="utf-8">\n      <script  nonce="DtuqBpgs6EkXwdaJIpfzGA==">\n        !function(){window.initErrorstack||(window.initErrorstack=[]),window.onerror=function(r,i,n,o,t){r.indexOf("Script error.")>-1||window.initErrorstack.push({errorMsg:r,url:i,lineNumber:n,column:o,errorObj:t})}}();\n      </script>\n    \n    \n  \n  <script id="bouncer_terminate_iframe" nonce="DtuqBpgs6EkXwdaJIpfzGA==">\n    if (window.top != window) {\n  window.top.postMessage({\'bouncer\': true, \'event\': \'complete\'}, \'*\');\n}\n  </script>\n  <script id="ttft_boot_data" nonce="DtuqBpgs6EkXwdaJIpfzGA==">\n    window.ttftData={"transaction_id":"00a8a6f100feee79.dc57abd515abaa27\\u003c:0056268e004fb941","server_request_start_time":1550877563428,"user_id":null,"is_ssl":true,"rendered_on_server":true,"is_tfe":true,"client":"macaw-swift","tfe_version":"tsa_b\\/1.0.1\\/201902

In [117]:
tree=lxml.etree.parse(StringIO(get_node(urldb['url'][1])),)


['https://pbs.twimg.com/profile_images/986439975871762432/0Y8zP4ir_400x400.jpg']